### CNN Model and Optimize using Keras Tuner

In [1]:
# Import libraries
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras import Sequential 

Using TensorFlow backend.


In [9]:
# Import Dataset
fashion_mnist = tf.keras.datasets.fashion_mnist

In [10]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

In [13]:
# Scaling down the images
train_images = train_images/255.0
test_images = test_images/255.0

In [16]:
train_images[0].shape

(28, 28)

In [17]:
len(train_images)

60000

In [18]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [19]:
train_images.shape

(60000, 28, 28, 1)

In [22]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_kernel', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [23]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [24]:
tuner_search = RandomSearch(build_model,
                            objective='val_accuracy',
                            max_trials=5,
                            directory='output1',
                            project_name="MnistFashion")

In [25]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 16m 17s]
val_accuracy: 0.9125000238418579

Best val_accuracy So Far: 0.9125000238418579
Total elapsed time: 01h 09m 51s
INFO:tensorflow:Oracle triggered exit


In [26]:
model = tuner_search.get_best_models(num_models=1)[0]

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 128)       1280      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        55344     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               3096688   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 3,154,442
Trainable params: 3,154,442
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 343s 203ms/step - loss: 0.1198 - accuracy: 0.9559 - val_loss: 0.2655 - val_accuracy: 0.9123
Epoch 5/10
1688/1688 [==============================] - 383s 227ms/step - loss: 0.0790 - accuracy: 0.9718 - val_loss: 0.3037 - val_accuracy: 0.9123
Epoch 6/10
1688/1688 [==============================] - 407s 241ms/step - loss: 0.0558 - accuracy: 0.9794 - val_loss: 0.3469 - val_accuracy: 0.9138
Epoch 7/10
1688/1688 [==============================] - 406s 241ms/step - loss: 0.0385 - accuracy: 0.9863 - val_loss: 0.3953 - val_accuracy: 0.9157
Epoch 8/10
1688/1688 [==============================] - 431s 255ms/step - loss: 0.0288 - accuracy: 0.9897 - val_loss: 0.4999 - val_accuracy: 0.9090
Epoch 9/10
1688/1688 [==============================] - 366s 217ms/step - loss: 0.0238 - accuracy: 0.9919 - val_loss: 0.5302 - val_accuracy: 0.9105
Epoch 10/10
1688/1688 [==============================] - 320s 189ms/step - loss: 0.0196 - accuracy: 0.9936 - val